In [1]:
from Cells import Cells
from MGFeatures import *
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import napari
from tifffile import imsave
import skimage.measure
import time
import seaborn as sns
import matplotlib.pyplot as plt
import cc3d

C:\Users\Gast-User\.conda\envs\MG\lib\site-packages\numba\core\decorators.py:253: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
def get_area(labels, labels_list): 
    areas = []
    for i in labels_list:
        area = np.count_nonzero(labels == i)
        areas.append(area)
    areas = np.array(areas)
    return areas

In [3]:
folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\cell_final'
heterochromatin_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\heterochromatin_final'
gold_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\gold_final'
lysosomes_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\lysosomes_final'
mitochondria_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\mitochondria_final'
nucleus_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\nucleus_final'
ER_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\er_final'
golgi_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\golgi_final'

In [4]:
cell_stack = np.zeros(shape = (5, 37440, 35168))
for i in range(0,5):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    binary_image = cell_image.astype(bool)
    cell_stack[i] = binary_image
cell_labels = cc3d.connected_components(cell_stack, connectivity=18)

In [120]:
features2d = pd.read_excel('total_features_v5.xlsx', index_col=0)

In [121]:
features2d

,cell_area,cell_aspect,cell_roundness,number_mito,mean_mito_aspect,std_mito_aspect,mean_mito_area,std_mito_area,mean_mito_roundness,std_mito_roundness,...,gold_area,nucleus_area,golgi_area,cytoplasm_area,ER_lengths,sl_num,x1,x2,y1,y2
0,567924,0.692308,0.505520,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,13744,281080,0,286844,5790.005387,0,3256,4084,29148,30344
1,523764,0.821293,0.602579,13,0.859884,0.146664,1620.307692,978.392507,0.844462,0.142653,...,7468,291312,1237,232452,9256.108314,0,3658,4710,28706,29570
2,29448,0.535000,0.234340,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,540,0,0,29448,452.752307,0,4720,5120,29274,29488
3,110076,0.555133,0.506560,3,0.954545,0.078730,945.333333,392.333192,0.991240,0.099072,...,1368,72748,0,37328,0.000000,0,6784,7310,30750,31042
4,99388,0.684647,0.544691,3,0.944444,0.096225,574.000000,228.085510,0.984248,0.111467,...,712,13240,0,86148,1835.581945,0,9622,10104,29474,29804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,369140,0.600406,0.483445,5,0.850536,0.065374,1480.800000,277.105034,0.761238,0.125115,...,49612,154772,0,214368,1810.378847,150,9782,10768,32244,32836
0,365204,0.560158,0.452241,3,0.951297,0.004712,1310.666667,246.619815,0.972257,0.014880,...,23112,173964,0,191240,942.607209,151,9792,10806,32336,32904
0,380196,0.542650,0.398616,2,0.917273,0.052712,1592.000000,175.362482,0.919702,0.064701,...,11752,189780,0,190416,524.724962,152,9754,10856,32274,32872
0,333256,0.579957,0.482261,2,0.797203,0.093951,1330.000000,82.024387,0.755895,0.221953,...,20952,187116,0,146140,1117.300639,153,9834,10772,32334,32878


In [128]:
features5 = pd.DataFrame()
features5 = features2d[(features2d['sl_num'] >= 0) & (features2d['sl_num']  < 5)].reset_index()

In [129]:
features5

,index,cell_area,cell_aspect,cell_roundness,number_mito,mean_mito_aspect,std_mito_aspect,mean_mito_area,std_mito_area,mean_mito_roundness,...,gold_area,nucleus_area,golgi_area,cytoplasm_area,ER_lengths,sl_num,x1,x2,y1,y2
0,0,567924,0.692308,0.505520,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,13744,281080,0,286844,5790.005387,0,3256,4084,29148,30344
1,1,523764,0.821293,0.602579,13,0.859884,0.146664,1620.307692,978.392507,0.844462,...,7468,291312,1237,232452,9256.108314,0,3658,4710,28706,29570
2,2,29448,0.535000,0.234340,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,540,0,0,29448,452.752307,0,4720,5120,29274,29488
3,3,110076,0.555133,0.506560,3,0.954545,0.078730,945.333333,392.333192,0.991240,...,1368,72748,0,37328,0.000000,0,6784,7310,30750,31042
4,4,99388,0.684647,0.544691,3,0.944444,0.096225,574.000000,228.085510,0.984248,...,712,13240,0,86148,1835.581945,0,9622,10104,29474,29804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,22,520156,0.772201,0.617055,1,0.607843,0.000000,4148.000000,0.000000,0.507631,...,10928,314584,0,205572,0.000000,4,17208,18244,30548,31348
133,23,82040,0.935294,0.903604,1,0.958333,0.000000,1808.000000,0.000000,0.999139,...,2920,20060,0,61980,422.617312,4,17266,17606,34182,34500
134,24,4,1.000000,1.273240,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,4,0.000000,4,17794,17796,7276,7278
135,25,245740,0.652174,0.577604,4,0.772139,0.133778,1058.000000,377.656811,0.733869,...,2100,124600,2241,121140,2015.162679,4,17796,18532,11714,12194


In [49]:
for i in range(173):
    if (features5['index'][i]) == 0:
        print(features5['x1'][i])

3256
3250
3242
3270
3256
3258


In [51]:
filtered_values = features5.loc[features5['sl_num'] == 0, 'x1']
print(filtered_values)

0      3256
1      3658
2      4720
3      6784
4      9622
5     10064
6     10106
7     10120
8     13274
9     13856
10    14194
11    14280
12    14425
13    14582
14    16262
15    16442
16    16994
17    17168
18    17252
19    17626
20    17828
21    17832
22    17930
Name: x1, dtype: int64


In [73]:
filtered = features5.loc[features5['sl_num'] == 0]
filtered['x1'][0]

3256

In [16]:
for i in range(1, cell_labels[0].max()+1):
    layer = np.where(cell_labels[0] == i)#coordinates of the first section of each of the cells
    if len(layer[0]) > 0 and len(layer[1]) > 0:
        filtered = features5.loc[features5['sl_num'] == 0] 
        id_3d_values = []
        for n in range(len(filtered)):
            if (layer[0].min() >= filtered['x1'][n]) and (layer[0].max() <= filtered['x2'][n]) and (layer[1].min() >= filtered['y1'][n]) and (layer[1].max() <= filtered['y2'][n]):
                id_3d_values.append(i)
        filtered['id_3d'] = id_3d_values
        features5.loc[filtered.index, 'id_3d'] = filtered['id_3d']

ValueError: Length of values (0) does not match length of index (23)

In [55]:
filtered = features5.loc[features5['sl_num'] == 2] 
features5new = features5.drop(columns = ['id_3d'])
features5new

,index,cell_area,cell_aspect,cell_roundness,number_mito,mean_mito_aspect,std_mito_aspect,mean_mito_area,std_mito_area,mean_mito_roundness,...,gold_area,nucleus_area,golgi_area,cytoplasm_area,ER_lengths,sl_num,x1,x2,y1,y2
0,0,567924,0.692308,0.505520,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,13744,281080,0,286844,5790.005387,0,3256,4084,29148,30344
1,1,523764,0.821293,0.602579,13,0.859884,0.146664,1620.307692,978.392507,0.844462,...,7468,291312,1237,232452,9256.108314,0,3658,4710,28706,29570
2,2,29448,0.535000,0.234340,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,540,0,0,29448,452.752307,0,4720,5120,29274,29488
3,3,110076,0.555133,0.506560,3,0.954545,0.078730,945.333333,392.333192,0.991240,...,1368,72748,0,37328,0.000000,0,6784,7310,30750,31042
4,4,99388,0.684647,0.544691,3,0.944444,0.096225,574.000000,228.085510,0.984248,...,712,13240,0,86148,1835.581945,0,9622,10104,29474,29804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,22,520156,0.772201,0.617055,1,0.607843,0.000000,4148.000000,0.000000,0.507631,...,10928,314584,0,205572,0.000000,4,17208,18244,30548,31348
133,23,82040,0.935294,0.903604,1,0.958333,0.000000,1808.000000,0.000000,0.999139,...,2920,20060,0,61980,422.617312,4,17266,17606,34182,34500
134,24,4,1.000000,1.273240,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,4,0.000000,4,17794,17796,7276,7278
135,25,245740,0.652174,0.577604,4,0.772139,0.133778,1058.000000,377.656811,0.733869,...,2100,124600,2241,121140,2015.162679,4,17796,18532,11714,12194


In [65]:
features5 = features5.drop(columns = ['id_3d'])
features

KeyError: "['id_3d'] not found in axis"

In [69]:
features5 = features5.drop(columns = ['index'])

In [70]:
features5

,cell_area,cell_aspect,cell_roundness,number_mito,mean_mito_aspect,std_mito_aspect,mean_mito_area,std_mito_area,mean_mito_roundness,std_mito_roundness,...,gold_area,nucleus_area,golgi_area,cytoplasm_area,ER_lengths,sl_num,x1,x2,y1,y2
0,567924,0.692308,0.505520,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,13744,281080,0,286844,5790.005387,0,3256,4084,29148,30344
1,523764,0.821293,0.602579,13,0.859884,0.146664,1620.307692,978.392507,0.844462,0.142653,...,7468,291312,1237,232452,9256.108314,0,3658,4710,28706,29570
2,29448,0.535000,0.234340,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,540,0,0,29448,452.752307,0,4720,5120,29274,29488
3,110076,0.555133,0.506560,3,0.954545,0.078730,945.333333,392.333192,0.991240,0.099072,...,1368,72748,0,37328,0.000000,0,6784,7310,30750,31042
4,99388,0.684647,0.544691,3,0.944444,0.096225,574.000000,228.085510,0.984248,0.111467,...,712,13240,0,86148,1835.581945,0,9622,10104,29474,29804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,520156,0.772201,0.617055,1,0.607843,0.000000,4148.000000,0.000000,0.507631,0.000000,...,10928,314584,0,205572,0.000000,4,17208,18244,30548,31348
133,82040,0.935294,0.903604,1,0.958333,0.000000,1808.000000,0.000000,0.999139,0.000000,...,2920,20060,0,61980,422.617312,4,17266,17606,34182,34500
134,4,1.000000,1.273240,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,4,0.000000,4,17794,17796,7276,7278
135,245740,0.652174,0.577604,4,0.772139,0.133778,1058.000000,377.656811,0.733869,0.150345,...,2100,124600,2241,121140,2015.162679,4,17796,18532,11714,12194


In [137]:
features5new =  features5.copy()
for j in range(5):
    filtered = pd.DataFrame()
    filtered = features5new.loc[features5new['sl_num'] == j] 
    id_3d_values = [0] * len(filtered)
    for i in range(1, cell_labels[j].max()+1):
        layer = np.where(cell_labels[j] == i)
        if layer[0].size != 0: #coordinates of the first section of each of the cells
            for n in range(len(filtered)):
                filtered_index = filtered.loc[filtered['index']==n]
                if (layer[0].min() >= filtered_index['x1'][0]) and (layer[0].max() <= filtered_index['x2'][0]) and (layer[1].min() >= filtered_index['y1'][0]) and (layer[1].max() <= filtered_index['y2'][0]):
                    id_3d_values[n] = i
    filtered['id_3d'] = id_3d_values
    features5new.loc[filtered.index, 'id_3d'] = filtered['id_3d']

KeyError: 0

In [143]:
features5new =  features5.copy()
features5new['id_3d'] = 0
for j in range(5):
    for i in range(1, cell_labels[j].max()+1):
        layer = np.where(cell_labels[j] == i)
        if layer[0].size != 0: #coordinates of the first section of each of the cells
            for n in range(len(features5new)):
                if features5new.loc[n, 'sl_num'] == j:
                    if (layer[0].min() >= features5new['x1'][n]) and (layer[0].max() <= features5new['x2'][n]) and (layer[1].min() >= features5new['y1'][n]) and (layer[1].max() <= features5new['y2'][n]):
                        features5new['id_3d'][n] = i

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_7084\1881578070.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features5new['id_3d'][n] = i


# Function features_to_3d

In [146]:
def features_to_3d(cell_labels, features):
    featuresnew =  features.copy()
    featuresnew['id_3d'] = 0
    for j in range(5):
        for i in range(1, cell_labels[j].max()+1):
            layer = np.where(cell_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(featuresnew)):
                    if featuresnew.loc[n, 'sl_num'] == j:
                        if (layer[0].min() >= featuresnew['x1'][n]) and (layer[0].max() <= featuresnew['x2'][n]) and (layer[1].min() >= featuresnew['y1'][n]) and (layer[1].max() <= featuresnew['y2'][n]):
                            featuresnew['id_3d'][n] = i

In [150]:
start_time = time.time()
cell_stack = np.zeros(shape = (5, 37440, 35168))
for i in range(0,5):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    binary_image = cell_image.astype(bool)
    cell_stack[i] = binary_image
cell_labels = cc3d.connected_components(cell_stack, connectivity=18)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

Elapsed time:  62.20431709289551


In [151]:
features2d = pd.read_excel('total_features_v5.xlsx', index_col=0)

In [152]:
features5 = pd.DataFrame()
features5 = features2d[(features2d['sl_num'] >= 0) & (features2d['sl_num']  < 5)].reset_index()

In [153]:
start_time = time.time()
features5_3d = features_to_3d(cell_labels, features5)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_7084\3347081662.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  featuresnew['id_3d'][n] = i


Elapsed time:  402.0276348590851


In [94]:
layers.to_excel('bbox_3d_0.xlsx')

In [20]:



layer2 = np.where(cell_labels[2]==1)
layer0[0].min(), layer1[0].min(), layer2[0].min() #x1, x2, y1, y2 coordinates of cell 1 in section 0


(3256, 3250, 3242)

In [ ]:
layer0 = np.where(cell_labels[0]==1)
    
layer1 = np.where(cell_labels[1]==1)

layer0[0].min(),layer0[0].max(), layer0[1].min(),  layer0[1].max(), layer1[0].min(),layer1[0].max(), layer1[1].min(),  layer1[1].max() #x1, x2, y1, y2 coordinates of cell 1 in section 0


In [ ]:
coords = list()
coords = np.where(cell_labels ==1)
coords_2 = np.where(cell_labels ==2)
coords, coords_2

In [61]:
m = cell_labels.max()
stack_coords = []
for i in range(1, m+1): #m+1, because python doesn´t include the last value 
    coords = []
    coords = list(np.where(cell_labels ==i))
    stack_coords.append(coords)
stack_coords

[[array([0, 0, 0, ..., 4, 4, 4], dtype=int64),
  array([3256, 3256, 3256, ..., 5091, 5091, 5091], dtype=int64),
  array([29756, 29757, 29758, ..., 29475, 29476, 29477], dtype=int64)],
 [array([0, 0, 0, ..., 4, 4, 4], dtype=int64),
  array([6784, 6784, 6784, ..., 7303, 7303, 7303], dtype=int64),
  array([30894, 30895, 30896, ..., 30925, 30926, 30927], dtype=int64)],
 [array([0, 0, 0, ..., 4, 4, 4], dtype=int64),
  array([ 9622,  9622,  9622, ..., 10089, 10089, 10089], dtype=int64),
  array([29706, 29707, 29708, ..., 29423, 29424, 29425], dtype=int64)],
 [array([0, 0, 0, ..., 4, 4, 4], dtype=int64),
  array([10064, 10064, 10064, ..., 10568, 10569, 10569], dtype=int64),
  array([32152, 32153, 32154, ..., 31989, 31988, 31989], dtype=int64)],
 [array([0, 0, 0, ..., 4, 4, 4], dtype=int64),
  array([10106, 10106, 10106, ..., 10585, 10585, 10585], dtype=int64),
  array([30790, 30791, 30792, ..., 31161, 31162, 31163], dtype=int64)],
 [array([0, 0, 0, ..., 4, 4, 4], dtype=int64),
  array([10120,

In [ ]:
cell_1_0 = np.where(stack_coords[0][0]==0)

In [ ]:
cell_1_0

In [ ]:
properties_3d_all = pd.DataFrame()
for i in range(len(stack_coords)):
    properties_3d_list = []
    for j in range(stack_coords[i][0].max()):
        properties_3d = pd.DataFrame()
        properties_3d['cell_id'] = i
        properties_3d['section'] = j
        properties_3d['section_area'] = np.count_nonzero(stack_coords[i][0] == j)
        properties_3d_list.append(properties_3d) 
    properties_3d_all = pd.concat(properties_3d_list, ignore_index=True)
properties_3d_all

In [ ]:
first_coords[0].min(), first_coords[0].max(), first_coords[1].min(), first_coords[1].max(), first_coords[2].min(), first_coords[2].max()


In [ ]:
sec_coords = np.where(cell_labels == 2)

In [ ]:
sec_coords

In [ ]:
z1 = sec_coords[0].min()
z2 = sec_coords[0].max()
x1 = sec_coords[1].min() 
x2 = sec_coords[1].max()
y1 = sec_coords[2].min()
y2 = sec_coords[2].max()

In [ ]:
bbox = [[x1, y1, z1], [x2, y1, z1], [x1, y2, z1], [x2, y2, z1],[x1, y1, z2], [x2, y1, z2], [x1, y2, z2], [x2, y2, z2]]

In [ ]:
bbox_2 = [x1,x2, y1, y2, z1, z2]
bbox_2

In [ ]:
D=max(bbox_2[1]-bbox_2[0],bbox_2[3]-bbox_2[2])
d=min(bbox_2[1]-bbox_2[0],bbox_2[3]-bbox_2[2]) #x and y: 10nm and z:200nm
area = D*d
volumen = D*d*(bbox_2[5]-bbox_2[4]*20)
D, d, D/d, area, volumen

In [ ]:
area_0 = np.where(sec_coords[0] == 0)
area_0

In [ ]:
volumen, (area_0*20 + area_1*20 + area_2*20 + area_3*20 +area_4*20+ area_5*20+ area_6*20)

In [ ]:
cell_area_1 = get_area(first_coords, 1)

In [ ]:
cell_area

In [ ]:
for i in range(35168):
    for j in range(37440):
        for a in range(7):
            if cell_labels_t[i][j][a] !=0:
                print(i), print(j), print(a)
    

In [ ]:
bbox_rects = []
for region in regionprops(cell_labels):
    bbox_rect = []
    bbox = region.bbox
    bbox_rect = draw_bbox(bbox)
    bbox_rects.append(bbox_rect)

In [ ]:
 bbox_rects

In [ ]:
bbox_rects = []
for r in regionprops(cell_labels):
    bbox = region.bbox
    

In [ ]:
def get_bbox(self, labels):
    self.bboxes = [[],[]]
    for region in regionprops(labels):
        bbox=region.bbox
        self.bboxes[0].append(bbox)
        bbox_rect = Cells.draw_bbox(bbox)
        self.bboxes[1].append(bbox_rect)


In [ ]:
cell_stack_labels = cc3d.connected_components(cell_stack, connectivity=26)

In [ ]:
help(cc3d.connected_components)

In [ ]:
cell_stack_labels.shape

In [ ]:
N = np.max(cell_stack_labels)

In [ ]:
N

In [ ]:
binary_image = cell_image.astype(bool)


In [ ]:
labels = np.random.randint(0, 2, size=(7, 37440, 35168))

In [ ]:
labels